In [ ]:
%matplotlib widget
# %matplotlib notebook
# %matplotlib inline

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from scipy.optimize import leastsq
from scipy.linalg import null_space

In [ ]:
def get_cylinder_points(p1, p2, r):
    ax_vec = p2 - p1
    h = np.linalg.norm(ax_vec)
    ax_vec = ax_vec / np.linalg.norm(ax_vec)
    ax_null = null_space(np.array([ax_vec]))
    axpts = np.linspace(0, h, 10)
    thetas = np.linspace(0, 2 * np.pi, 20)
    axpts, thetas = np.meshgrid(axpts, thetas)
    X, Y, Z = [p1[i] + ax_vec[i] * axpts + r * np.sin(thetas) * ax_null[i, 0] + r * np.cos(thetas) * ax_null[i, 1] for i in [0, 1, 2]]
    return X, Y, Z

In [ ]:
X, Y, Z = get_cylinder_points(np.array([0, 0, 0]), np.array([1, 1, 1]), 0.5)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, Z)

In [ ]:
def cylinderFitting(xyz,p,th):

    """
    This is a fitting for a vertical cylinder fitting
    Reference:
    http://www.int-arch-photogramm-remote-sens-spatial-inf-sci.net/XXXIX-B5/169/2012/isprsarchives-XXXIX-B5-169-2012.pdf

    xyz is a matrix contain at least 5 rows, and each row stores x y z of a cylindrical surface
    p is initial values of the parameter;
    p[0] = Xc, x coordinate of the cylinder centre
    P[1] = Yc, y coordinate of the cylinder centre
    P[2] = alpha, rotation angle (radian) about the x-axis
    P[3] = beta, rotation angle (radian) about the y-axis
    P[4] = r, radius of the cylinder

    th, threshold for the convergence of the least squares

    """   
    x = xyz[:,0]
    y = xyz[:,1]
    z = xyz[:,2]

    fitfunc = lambda p, x, y, z: (- np.cos(p[3])*(p[0] - x) - z*np.cos(p[2])*np.sin(p[3]) - np.sin(p[2])*np.sin(p[3])*(p[1] - y))**2 + (z*np.sin(p[2]) - np.cos(p[2])*(p[1] - y))**2 #fit function
    errfunc = lambda p, x, y, z: fitfunc(p, x, y, z) - p[4]**2 #error function 

    est_p , success = leastsq(errfunc, p, args=(x, y, z), maxfev=1000)

    return est_p

H = 5
R = 1
n = 100
xyz = np.zeros((n, 3))

for i in range(n):
    r = np.random.normal(R, 0.1)
    theta = np.random.uniform(0, 2 * np.pi)
    z = np.random.uniform(0, H)
    xyz[i, :] = [r * np.cos(theta), r * np.sin(theta), z]
print("Initial Parameters: ")
p = np.array([-13.79,-8.45,0,0,0.3])
print(p)
print(" ")

print("Performing Cylinder Fitting ... ")
est_p =  cylinderFitting(xyz,p,0.00001)
print("Fitting Done!")
print(" ")


print("Estimated Parameters: ")
print(est_p)